# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.264952278992971)),
 (1, np.float64(2.264952278992971)),
 (2, np.float64(2.264952278992971)),
 (3, np.float64(2.264952278992971)),
 (4, np.float64(2.264952278992971))]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2)]),
 (1, [('banana', 2), ('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.033343945550146636)),
   (None, np.float64(0.05906513548377046)),
   (None, np.float64(0.06262900124422388)),
   (None, np.float64(0.09594255882910163)),
   (None, np.float64(0.11820244273241975)),
   (None, np.float64(0.13410906358562147)),
   (None, np.float64(0.14508296211616245)),
   (None, np.float64(0.15671310620080114)),
   (None, np.float64(0.3526399253774174)),
   (None, np.float64(0.39133037120251657)),
   (None, np.float64(0.408903262856544)),
   (None, np.float64(0.4312082210670848)),
   (None, np.float64(0.43499456315058116)),
   (None, np.float64(0.4397359428282247)),
   (None, np.float64(0.4763238066720089))]),
 (1,
  [(None, np.float64(0.5017106560082311)),
   (None, np.float64(0.5197672739592133)),
   (None, np.float64(0.5511590948403385)),
   (None, np.float64(0.5701806238342352)),
   (None, np.float64(0.6045480666335)),
   (None, np.float64(0.6385377608212646)),
   (None, np.float64(0.6406462489939784)),
   (None, np.float64(0.696028244307

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
arr = np.random.uniform(0, 20, 20)
def RECORDREADER():
  return [(id, i) for id, i in enumerate(arr)]

def MAP(_, x):
  yield (None, x)

def REDUCE(_, nums):
  m = -np.inf
  for x in nums:
    m = max(m, x)
  yield m

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)
arr.max()

[np.float64(19.56286991485948)]


np.float64(19.56286991485948)

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
arr = np.random.uniform(0, 20, 20)
def RECORDREADER():
  return [(id, i) for id, i in enumerate(arr)]

def MAP(_, x):
  yield (None, x)

def REDUCE(_, nums):
  m = 0
  count = 0
  for x in nums:
    m += x
    count += 1
  yield m/count

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)
np.average(arr)

[np.float64(10.121593780144444)]


np.float64(10.121593780144444)

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

In [ ]:
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

In [ ]:
def groupbykey(iterable):
    # Сортировка по ключу
    iterable = sorted(iterable, key=lambda x: x[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [ ]:
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('a', 1), ('banana', 1), ('is', 9), ('it', 9), ('what', 5)]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=51, gender='male', social_contacts=20),
    User(id=0, age=95, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=35, gender='female', social_contacts=240),
    User(id=1, age=45, gender='female', social_contacts=240),
    User(id=2, age=55, gender='male', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=75, gender='female', social_contacts=240),
    User(id=4, age=40, gender='male', social_contacts=110)
]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

In [ ]:
maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for id, user in enumerate(split):
            yield (id, user)

    # распределяем по вычислительным узлам
    split_size = len(input_collection) // maps
    r = len(input_collection) % maps
    ids = 0
    for i in range(maps):
        if r > 0:
            r -= 1
            yield RECORDREADER(input_collection[ids : ids + split_size + 1])
            ids += split_size + 1
        else:
            yield RECORDREADER(input_collection[ids : ids + split_size])
            ids += split_size


# Группировка пользователей по id
def MAP(_, user: str):
    yield (user.id, user)


# убираем повторяющиеся по id
def REDUCE(user_id: int, users: Iterator[int]):
    yield users[0]


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

10 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=55, social_contacts=20, gender='male'),
   User(id=2, age=55, social_contacts=500, gender='male'),
   User(id=4, age=40, social_contacts=110, gender='male')]),
 (1,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
def C(row: NamedTuple):
  return row.gender == 'female'

def MAP(_, row: NamedTuple):
    if C(row):
        yield (row, row)


def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

5 key-value pairs were sent over a network.


[(0,
  [(User(id=1, age=45, social_contacts=240, gender='female'),
    [User(id=1, age=45, social_contacts=240, gender='female')]),
   (User(id=3, age=33, social_contacts=800, gender='female'),
    [User(id=3, age=33, social_contacts=800, gender='female')])]),
 (1,
  [(User(id=1, age=25, social_contacts=240, gender='female'),
    [User(id=1, age=25, social_contacts=240, gender='female')]),
   (User(id=1, age=35, social_contacts=240, gender='female'),
    [User(id=1, age=35, social_contacts=240, gender='female')]),
   (User(id=3, age=75, social_contacts=240, gender='female'),
    [User(id=3, age=75, social_contacts=240, gender='female')])])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
input_collection = [
    dict(id=0, age=55, gender='male', social_contacts=20),
    dict(id=0, age=51, gender='male', social_contacts=20),
    dict(id=0, age=95, gender='male', social_contacts=20),
    dict(id=1, age=25, gender='female', social_contacts=240),
    dict(id=1, age=35, gender='female', social_contacts=240),
    dict(id=1, age=45, gender='female', social_contacts=240),
    dict(id=2, age=55, gender='male', social_contacts=500),
    dict(id=3, age=33, gender='female', social_contacts=800),
    dict(id=3, age=75, gender='female', social_contacts=240),
    dict(id=4, age=40, gender='male', social_contacts=110)
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

# Группировка по id
def MAP(_, row: dict):
    S = ['id', 'age']
    row = row.copy()
    for t in list(row.keys()):
        if t not in S:
            del row[t]
    yield(row['id'], row)


def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows[0])


def RECORDREADER():
    return [(u["id"], u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, {'id': 0, 'age': 55}),
 (1, {'id': 1, 'age': 25}),
 (2, {'id': 2, 'age': 55}),
 (3, {'id': 3, 'age': 33}),
 (4, {'id': 4, 'age': 40})]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
input_collection_a = [
    User(id=1, age=42, gender="female", social_contacts=154),
    User(id=2, age=54, gender="male", social_contacts=542),
    User(id=3, age=24, gender="male", social_contacts=11),
    User(id=4, age=11, gender="female", social_contacts=542),
    User(id=5, age=99, gender="female", social_contacts=54),
]

input_collection_b = [
    User(id=5, age=99, gender="female", social_contacts=54),
    User(id=6, age=2, gender="female", social_contacts=52),
    User(id=7, age=13, gender="male", social_contacts=21),
]

# Группировка пользователей по id
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)



def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=1, age=42, social_contacts=154, gender='female'),
  User(id=1, age=42, social_contacts=154, gender='female')),
 (User(id=2, age=54, social_contacts=542, gender='male'),
  User(id=2, age=54, social_contacts=542, gender='male')),
 (User(id=3, age=24, social_contacts=11, gender='male'),
  User(id=3, age=24, social_contacts=11, gender='male')),
 (User(id=4, age=11, social_contacts=542, gender='female'),
  User(id=4, age=11, social_contacts=542, gender='female')),
 (User(id=5, age=99, social_contacts=54, gender='female'),
  User(id=5, age=99, social_contacts=54, gender='female')),
 (User(id=6, age=2, social_contacts=52, gender='female'),
  User(id=6, age=2, social_contacts=52, gender='female')),
 (User(id=7, age=13, social_contacts=21, gender='male'),
  User(id=7, age=13, social_contacts=21, gender='male'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
input_collection_a = [
    User(id=1, age=42, gender="female", social_contacts=154),
    User(id=2, age=54, gender="male", social_contacts=542),
    User(id=3, age=24, gender="male", social_contacts=11),
    User(id=4, age=11, gender="female", social_contacts=542),
    User(id=5, age=99, gender="female", social_contacts=54),
]

input_collection_b = [
    User(id=5, age=99, gender="female", social_contacts=54),
    User(id=6, age=2, gender="female", social_contacts=52),
    User(id=7, age=13, gender="male", social_contacts=21),
]

# Группировка пользователей по id
def MAP(_, row: NamedTuple):
    yield (row.id, row)



def REDUCE(row: str, rows: Iterator[NamedTuple]):
    if len(rows) > 1:
        yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=5, age=99, social_contacts=54, gender='female'),
  User(id=5, age=99, social_contacts=54, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
input_collection_a = [
    User(id=1, age=42, gender="female", social_contacts=154),
    User(id=2, age=54, gender="male", social_contacts=542),
    User(id=3, age=24, gender="male", social_contacts=11),
    User(id=4, age=11, gender="female", social_contacts=542),
    User(id=5, age=99, gender="female", social_contacts=54),
]

input_collection_b = [
    User(id=5, age=99, gender="female", social_contacts=54),
    User(id=6, age=2, gender="female", social_contacts=52),
    User(id=7, age=13, gender="male", social_contacts=21),
]

# Группировка пользователей по id
def MAP(id: int, row: NamedTuple):
    yield (row, id)



def REDUCE(user, ids):
    if ids == [0]:
        yield user


def RECORDREADER():
    return [(0, u) for u in input_collection_a] + [(1, u) for u in input_collection_b]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[User(id=1, age=42, social_contacts=154, gender='female'),
 User(id=2, age=54, social_contacts=542, gender='male'),
 User(id=3, age=24, social_contacts=11, gender='male'),
 User(id=4, age=11, social_contacts=542, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
input_collection_a = [
    dict(id=0, age=55, gender='male', social_contacts=20),
    dict(id=1, age=51, gender='male', social_contacts=20),
    dict(id=2, age=95, gender='male', social_contacts=20),
    dict(id=3, age=25, gender='female', social_contacts=240),
    dict(id=4, age=35, gender='female', social_contacts=240)
]

input_collection_b = [
    dict(id=1, country='Russia'),
    dict(id=2, country='USA'),
    dict(id=3, country='UK'),
    dict(id=4, country='France'),
    dict(id=0, country='China')
]

def MAP(id: int, row: dict):
    yield (row['id'], row)



def REDUCE(id, rows):
    combined = {}
    for row in rows:
        combined.update(row)
    yield combined


def RECORDREADER():
    return [(0, u) for u in input_collection_a] + [(1, u) for u in input_collection_b]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output



[{'id': 0,
  'age': 55,
  'gender': 'male',
  'social_contacts': 20,
  'country': 'China'},
 {'id': 1,
  'age': 51,
  'gender': 'male',
  'social_contacts': 20,
  'country': 'Russia'},
 {'id': 2,
  'age': 95,
  'gender': 'male',
  'social_contacts': 20,
  'country': 'USA'},
 {'id': 3,
  'age': 25,
  'gender': 'female',
  'social_contacts': 240,
  'country': 'UK'},
 {'id': 4,
  'age': 35,
  'gender': 'female',
  'social_contacts': 240,
  'country': 'France'}]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
input_collection = [
    dict(id=0, age=55, gender='male', social_contacts=20),
    dict(id=1, age=51, gender='male', social_contacts=20),
    dict(id=2, age=95, gender='male', social_contacts=20),
    dict(id=3, age=25, gender='female', social_contacts=240),
    dict(id=4, age=35, gender='female', social_contacts=240),
    dict(id=5, age=45, gender='female', social_contacts=240),
    dict(id=6, age=55, gender='male', social_contacts=500),
    dict(id=7, age=33, gender='female', social_contacts=800),
    dict(id=8, age=75, gender='female', social_contacts=240),
    dict(id=9, age=40, gender='male', social_contacts=110)
]

def MAP(id: int, row: dict):
    yield (row['gender'], row['age'])



def REDUCE(gender, ages):
    average_age = sum(ages) / len(ages)
    yield dict(gender=gender, average_age=average_age)


def RECORDREADER():
    return [(u['id'], u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[{'gender': 'male', 'average_age': 59.2},
 {'gender': 'female', 'average_age': 42.6}]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [ ]:
import numpy as np
I = 5
J = 4
K = 4
matr = np.random.rand(I, J)
vec = np.random.rand(J)

def RECORDREADER():
    for j in range(matr.shape[1]):
        yield (j, vec[j])

def MAP(j, v1):
    for i in range(matr.shape[0]):
        v2 = matr[i, j] * v1
        yield (i, v2)


def REDUCE(i, values):
    v1 = sum(values)
    yield (v1)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = np.array(list(output))
output

array([1.32051843, 1.76820404, 0.71746271, 1.66381213, 0.91505739])

In [ ]:
matr @ vec

array([1.32051843, 1.76820404, 0.71746271, 1.66381213, 0.91505739])

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
    (j, k) = k1
    w = v1
    # solution code that yield(k2,v2) pairs
    for i in range(small_mat.shape[0]):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield(k2, v2)

def REDUCE(key, values):
  (i, k) = key
  v1 = sum(values)
  yield(key, v1)

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(I):
    for k in range(K):
      for j in range(J):
        yield (((i,j), small_mat[i,j]), ((j,k), big_mat[j, k]))

def MAP(pair1, pair2):
    (i, j), w1 = pair1
    (j, k), w2 = pair2
    yield((i, k), w1 * w2)

def REDUCE(key, values):
  v1 = sum(values)
  yield(key, v1)

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)
def INPUTFORMAT():
    global maps

    def RECORDREADER(I_range):
      for i in I_range:
        for k in range(K):
          for j in range(J):
            yield (((i,j), small_mat[i,j]), ((j,k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
      yield RECORDREADER(range(i, i + split_size))

def MAP(pair1, pair2):
    (i, j), w1 = pair1
    (j, k), w2 = pair2
    yield((i, k), w1 * w2)

def REDUCE(key, values):
  v1 = sum(values)
  yield(key, v1)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

reference_solution = small_mat @ big_mat
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Будет работать, так как каждая пара однозначно идентифицируется по индексам